In [ ]:
%AddDeps org.apache.spark spark-sql-kafka-0-10_2.11 2.4.3 --transitive 
%AddDeps org.apache.spark spark-streaming-kafka-0-10_2.11 2.4.3 --transitive
%AddDeps org.apache.spark spark-avro_2.11 2.4.3 --transitive

In [ ]:
import org.apache.spark.sql.types._
import org.apache.spark.sql.functions._

val base_path = "..."

val broker = "..."

val spark = SparkSession
  .builder
  .appName("pm")
  .getOrCreate()

val input_warnings = spark
  .readStream
  .format("kafka")
  .option("kafka.bootstrap.servers", broker + ":9092")
  .option("subscribe", "warnings-simple")
  .load()

val raw_warnings = input_warnings.selectExpr("CAST(value AS STRING)")

val warning_schema = new StructType()
      .add("deviceID",StringType)
      .add("timestamp",StringType)
      .add("warning",StringType)

val warnings = raw_warnings.select(from_json(col("value").cast("string"), warning_schema).as("data"))
    .select("data.*")

warnings
  .writeStream
  .outputMode("append")
  .format("parquet")
  .option("checkpointLocation", base_path + "checkpoint/warnings/")
  .option("path", base_path + "warnings.parquet")
  .start()
  .awaitTermination()